In [ ]:
import sys

sys.path.insert(0, '/mnt/data/molchanov/dltranz')

import torch
import torch.nn.functional as F
from torch.autograd import Function, Variable
import numpy as np
import random

import matplotlib.pyplot as plt

from domyshnik.models import *
from domyshnik.data import *
from domyshnik.constants import *
from domyshnik.utils import *

def draw(imgs):
        if isinstance(imgs, list):
            imgs = torch.stack(imgs)
        fig = plt.figure()
        rows, columns = 1, imgs.shape[0]
        for i in range(imgs.shape[0]):
            fig.add_subplot(rows, columns, i+1)
            if imgs[i].size(0) == 3:
                plt.imshow(imgs[i].transpose(0, 1).transpose(1, 2))
            else:
                plt.imshow(imgs[i])
        plt.show()
        
%matplotlib inline


In [ ]:
los = ClusterisationLoss(margin=0.5, 
                         input_dim=128, 
                         num_classes=10, 
                         device='cpu')

t = torch.randn(120, 128)
y, z = los(t)
(y + z).backward()

In [ ]:
class GambelSoftmax(nn.Module):
    
    def __init__(self, k, t):
        super().__init__()
        self.k = k
        self.t = t
        
        
    '''
    x - logit probs
    '''
    def forward(self, x):
        u = torch.rand(self.k)
        g = -1*torch.log(-1*torch.log(u))
        return F.softmax((x + g)/t, dim=-1)
        

In [ ]:
K, t = 10, 0.1
g_softmax = GambelSoftmax(K, t)

p = torch.log(torch.rand(K))
p

In [ ]:
p, g_softmax(p)

In [ ]:
class MyFunction(Function):
        

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight)
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        print(grad_output.size())
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        grad_input = grad_output.mm(weight.t())
        grad_weight = grad_output.t().mm(input)
        grad_bias = grad_output.sum(0)

        return grad_input, grad_weight, grad_bias

In [ ]:
inp, w, b = torch.randn(10, 128), torch.randn(128, 64), torch.Tensor([-1.0])
inp.requires_grad=True
w.requires_grad=True
b.requires_grad=True

In [ ]:
fnc = MyFunction.apply
y = fnc(inp, w, b)
z = y.sum()

z.backward()

In [ ]:
# softmax grad
batch_probs = torch.log(torch.rand(3, 10))
batch_probs

In [ ]:
s = batch_probs

In [ ]:
torch.einsum